In [18]:
%load_ext autoreload
%autoreload 2
import os
import sys

user_home = os.path.expanduser('~')

if '/home' in user_home:  # UNI env
    # set softlab_path, project_path and 
    # other useful variables used in the notebook
    with open(os.path.join(user_home, 
                           'WYM_Test', 'wym_github', 'wym', 
                           'import_utility.py')) as env_utility:
        exec(env_utility.read())

else:  # Google Colab remote environment
    from google.colab import drive
    drive.mount('/content/drive')
    !pip install lime
    # !pip install -q spacy
    !pip install -q pytorch-lightning
    !pip install -q transformers
    !pip install -q -U sentence-transformers

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload

Currently executing on sparc20
Jupyter Notebook detected.


In [4]:
#@title Counterfactual examples
from wym.run_experiments.wym_evaluation import WYMEvaluation
from Landmark_github.evaluation.Evaluate_explanation_Batch import *

# init of WYM, Routine (tokenization, BERT -> embeddings, 
# word pairing w/ stable marriage -> decision unit, 
# DL network -> relevance score, 
# features as numeric values -> ML on decision units + relevance scores -> prediction,
# reverse of feature function -> impacts, explanation) 
dataset_name = "iTunes-Amazon" # @param ['BeerAdvo-RateBeer', 'fodors-zagats', 'iTunes-Amazon', 'dirty_itunes_amazon', 'DBLP-Scholar', 'dirty_dblp_scholar', 'walmart-amazon', 'dirty_walmart_amazon', 'DBLP-ACM', 'dirty_dblp_acm', 'Abt-Buy', 'Amazon-Google']
print(f"Using dataset {dataset_name}.")
reset_files = False #@param ["True", "False"] {type:"raw"}
reset_networks = False #@param ["True", "False"] {type:"raw"}
we_finetuned = True #@param ["True", "False"] {type:"raw"}
sentence_embedding = False  #@param ["True", "False"] {type:"raw"}
batch_size = 256 #@param {type:"slider", min:64, max:1024, step:64}
delta = .1 #@param {type:"slider", min:0.1, max:0.9, step:0.1}
# landmark tuning for DU, default: True, True
evaluate_removing_du = True #@param ["True", "False"] {type:"raw"}
recompute_embeddings = True #@param ["True", "False"] {type:"raw"}

utility = True #@param ["True", "False", "\"degradation\"", "\"single_units\"", "\"incremental\"", "\"all\"", "\"AOPC\"", "\"sufficiency\""] {type:"raw"}
# number of casual perturbations 
k = 10 #@param {type:"slider", min:1, max:10, step:1}
evaluate_positive = False #@param ["True", "False", "\"all\""] {type:"raw"}

variable_side = "all" #@param ["all", "left", "right"]
fixed_side = "all" #@param ["all", "left", "right"]

# evaluate WYM evaluation with Landmark
wym_ev = WYMEvaluation(dataset_name, evaluate_removing_du=evaluate_removing_du,
                       recompute_embeddings=recompute_embeddings, 
                       variable_side=variable_side, fixed_side=fixed_side, 
                       reset_files=reset_files, reset_networks=reset_networks, 
                       we_finetuned=we_finetuned, 
                       sentence_embedding=sentence_embedding, batch_size=batch_size,
                       delta=delta)

result_objects = wym_ev.evaluate_single_df(utility=utility, explanation='last', evaluate_positive=evaluate_positive)

if evaluate_positive == 'all':
    pos_res_df, pos_ev_expl, neg_res_df, neg_ev_expl = result_objects
    display(pos_res_df)

elif evaluate_positive:
    pos_res_df, pos_ev_expl = result_objects
    display(pos_res_df)
else:
    neg_res_df, neg_ev_expl = result_objects
    display(neg_res_df)

Using dataset iTunes-Amazon.
Loaded /home/giacomo.guiduzzi/content/drive/Shareddrives/SoftLab/Projects/WYM/dataset_files/iTunes-Amazon/BERT/words_maps.pickle
Loaded embeddings.
Loaded word pairs.


  0%|          | 0/13 [00:00<?, ?it/s]

LR        -f1 1.0
LR_std    -f1 1.0
LDA       -f1 0.7142857142857143
LDA_std   -f1 0.7142857142857143
KNN       -f1 0.8679245283018868
CART      -f1 0.9090909090909091
NB        -f1 0.7042253521126761
SVM       -f1 0.9811320754716981
AB        -f1 0.9818181818181818
GBM       -f1 0.9259259259259259
RF        -f1 0.9811320754716981
ET        -f1 0.9629629629629629
dummy     -f1 0.16666666666666666


train                   valid                    test            \
               f1 precision recall     f1 precision recall     f1 precision   
model_name                                                                    
LR         1.0000    1.0000 1.0000 0.9231    0.9600 0.8889 1.0000    1.0000   
LR_std     1.0000    1.0000 1.0000 0.9091    0.8929 0.9259 1.0000    1.0000   
LDA        1.0000    1.0000 1.0000 0.7576    0.6410 0.9259 0.7143    0.6897   
LDA_std    1.0000    1.0000 1.0000 0.7576    0.6410 0.9259 0.7143    0.6897   
KNN        0.9548    0.9610 0.9487 0.9020    0.9583 0.8519 0.8679    0.8846   
CART       1.0000    1.0000 1.0000 0.8889    0.8889 0.8889 0.9091    0.8929   
NB         0.8556    0.7549 0.9872 0.7246    0.5952 0.9259 0.7042    0.5682   
SVM        0.9936    0.9873 1.0000 0.9231    0.9600 0.8889 0.9811    1.0000   
AB         1.0000    1.0000 1.0000 0.9434    0.9615 0.9259 0.9818    0.9643   
GBM        1.0000    1.0000 1.0000 0.9091    0.8929 0.9259 0.9259    0.9259   
RF         1.0000    1.0000 1.0000 0.9091    0.8929 0.9259 0.9811    1.0000   
ET         1.0000    1.0000 1.0000 0.8929    0.8621 0.9259 0.9630    0.9630   
dummy      0.1842    0.1892 0.1795 0.1667    0.1905 0.1481 0.1667    0.1905   

                   
           recall  
model_name         
LR         1.0000  
LR_std     1.0000  
LDA        0.7407  
LDA_std    0.7407  
KNN        0.8519  
CART       0.9259  
NB         0.9259  
SVM        0.9630  
AB         1.0000  
GBM        0.9259  
RF         0.9630  
ET         0.9630  
dummy      0.1481

Evaluating data.
Testing unit removal with -- token_contribution
Evaluating negative examples.
Mapping word to attr


...

Computing embedding


  0%|          | 0/19 [00:00<?, ?it/s]

generating word_pairs


  0%|          | 0/4633 [00:00<?, ?it/s]

,id,start_pred,comb_name,new_pred,correct,expected_delta,detected_delta,num_tokens,tokens_removed,error,conf
0,0,0.0000,change_class,0.0000,False,-2.3748,-0.0000,55,"[[A03_timbaland], [[UNP]]]",-2.3748,bert
1,0,0.0000,single_word,0.0000,True,1.6253,0.0000,55,"[[A02_.], [I03_.]]",1.6253,bert
2,0,0.0000,single_word,0.0000,True,1.1973,-0.0000,55,"[[A01_feat], [I02_feat]]",1.1973,bert
3,0,0.0000,single_word,0.0000,True,0.9229,0.0000,55,"[[G00_3:55], [O00_3:17]]",0.9229,bert
4,0,0.0000,single_word,0.0000,True,0.5384,0.0000,55,"[[C00_mail], [K00_mail]]",0.5384,bert
...,...,...,...,...,...,...,...,...,...,...,...
4628,108,0.0366,single_word,0.0463,False,-1.8949,-0.0098,27,"[[H02_15], [[UNP]]]",-1.8851,bert
4629,108,0.0366,single_word,0.7678,True,-4.1860,-0.7312,27,"[[A01_outro], [[UNP]]]",-3.4548,bert
4630,108,0.0366,all_opposite,0.9996,True,-11.1520,-0.9630,27,"[[F01_kidinakorner, [UNP], F03_records, [UNP], F00_2015, [UNP], B01_ambassadors, D01_music, F02_interscope, D00_alternative, D...",-10.1890,bert
4631,108,0.0366,change_class_D.10,0.7678,True,-4.1860,-0.7312,27,"[[A01_outro], [[UNP]]]",-3.4548,bert


## Show positive examples

In [5]:
pos_ev_expl.generate_counterfactual_examples()
display(pos_ev_expl.counterfactual_examples)
display(pos_ev_expl.best_counterfactuals)
# display(pos_ev_expl.counterfactuals_plotting_data)

NameError: ignored

In [ ]:
pos_ev_expl.counterfactuals_plotting_data

In [ ]:
original_cpd = pos_ev_expl.counterfactuals_plotting_data

In [ ]:
cpd_subset = original_cpd.loc[[2]]
# cpd_subset = original_cpd
pos_ev_expl.counterfactuals_plotting_data = cpd_subset
HTML(pos_ev_expl.plot_counterfactual())

In [ ]:
pos_ev_expl.best_counterfactuals.iloc[[7]]['tokens_removed'].values

In [ ]:
pos_ev_expl.best_counterfactuals.iloc[[7]]

In [ ]:
pos_ev_expl.dataset[pos_ev_expl.dataset['left_Song_Name'] == 'x2cu']

In [ ]:
pos_ev_expl.impacts_df.loc[pos_ev_expl.impacts_df['id'] == 25, :].sort_values('impact', ascending=False)

In [ ]:
unpaired = pd.concat([pos_ev_expl.impacts_df.loc[pos_ev_expl.impacts_df['left_word_prefixes'] == '[UNP]'], pos_ev_expl.impacts_df.loc[pos_ev_expl.impacts_df['right_word_prefixes'] == '[UNP]']])

# unpaired[unpaired.id.isin(pos_ev_expl.best_counterfactuals.id)]
# unpaired[unpaired.id == 25].loc[[678],:]
display(unpaired)

In [ ]:
pos_res_df[pos_res_df.id == 25]

In [ ]:
pos_ev_expl.dataset[pos_ev_expl.dataset['id'] == 25]

In [ ]:
wym_ev.word_relevance_df[wym_ev.word_relevance_df.id == 25]

## Show negative examples

In [6]:
neg_ev_expl.generate_counterfactual_examples()
display(neg_ev_expl.counterfactual_examples)
display(neg_ev_expl.best_counterfactuals)
display(neg_ev_expl.counterfactuals_plotting_data)

,id,start_pred,comb_name,new_pred,correct,expected_delta,detected_delta,num_tokens,tokens_removed,error,conf,left_id,right_id,label,left_Song_Name,left_Artist_Name,left_Album_Name,left_Genre,left_Price,left_CopyRight,left_Time,left_Released,right_Song_Name,right_Artist_Name,right_Album_Name,right_Genre,right_Price,right_CopyRight,right_Time,right_Released,left_description,right_description,encoded_descs
0,75,0.9787,single_word,0.2216,False,0.1957,0.7572,54,"[[C09_how], [K09_how]]",-0.5615,bert,5747,21508,0,dayum baby feat . sarah buxton,florida georgia line,here 's to the good times ... this is how we roll deluxe version,country music urban cowboy honky tonk contemporary country,$ 1.29,an 2013 big machine label group llc,3:26,25 nov 13,this is how we roll feat . luke bryan,florida georgia line,here 's to the good times ... this is how we roll,country,$ 1.29,( c 2013 big machine label group llc,3:40,november 25 2013,dayum baby feat . sarah buxton florida georgia line here 's to the good times ... this is how we roll deluxe version country m...,this is how we roll feat . luke bryan florida georgia line here 's to the good times ... this is how we roll country $ 1.29 ( ...,"[[A00_dayum, A01_baby, A02_feat, A03_., A04_sarah, A05_buxton, B00_florida, B01_georgia, B02_line, C00_here, C01_'s, C02_to, C..."


,id,start_pred,comb_name,new_pred,correct,expected_delta,detected_delta,num_tokens,tokens_removed,error,conf
3253,75,0.9787,single_word,0.2216,False,0.1957,0.7572,54,"[[C09_how], [K09_how]]",-0.5615,bert
3254,75,0.9787,single_word,0.2414,False,0.1867,0.7374,54,"[[C07_this], [K07_this]]",-0.5507,bert
3289,75,0.9787,all_opposite,0.0000,True,16.4245,0.9787,54,"[[A03_., C11_roll, G00_3:26, A02_feat, C09_how, C07_this, C08_is, C10_we, H00_25, F00_an, H01_nov, C11_roll, E01_1.29, B01_geo...",15.4457,bert


,id,start_pred,comb_name,new_pred,correct,expected_delta,detected_delta,num_tokens,tokens_removed,error,conf
3253,75,0.9787,single_word,0.2216,False,0.1957,0.7572,54,"[[C09_how], [K09_how]]",-0.5615,bert


,id,start_pred,comb_name,new_pred,correct,expected_delta,detected_delta,num_tokens,tokens_removed,error,conf,left_id,right_id,label,left_Song_Name,left_Artist_Name,left_Album_Name,left_Genre,left_Price,left_CopyRight,left_Time,left_Released,right_Song_Name,right_Artist_Name,right_Album_Name,right_Genre,right_Price,right_CopyRight,right_Time,right_Released,left_description,right_description,encoded_descs
0,75,0.9787,single_word,0.2216,False,0.1957,0.7572,54,"[[C09_how], [K09_how]]",-0.5615,bert,5747,21508,0,dayum baby feat . sarah buxton,florida georgia line,here 's to the good times ... this is how we roll deluxe version,country music urban cowboy honky tonk contemporary country,$ 1.29,an 2013 big machine label group llc,3:26,25 nov 13,this is how we roll feat . luke bryan,florida georgia line,here 's to the good times ... this is how we roll,country,$ 1.29,( c 2013 big machine label group llc,3:40,november 25 2013,dayum baby feat . sarah buxton florida georgia line here 's to the good times ... this is how we roll deluxe version country m...,this is how we roll feat . luke bryan florida georgia line here 's to the good times ... this is how we roll country $ 1.29 ( ...,"[[A00_dayum, A01_baby, A02_feat, A03_., A04_sarah, A05_buxton, B00_florida, B01_georgia, B02_line, C00_here, C01_'s, C02_to, C..."


In [7]:
neg_ev_expl.counterfactuals_plotting_data = neg_ev_expl.counterfactuals_plotting_data # [0:3]
HTML(neg_ev_expl.plot_counterfactual(beautify_table=False, align_left=False))

In [19]:
# barro le parole della canzone 1 e 2 (del titolo o del genere musicale della canzone 1)
def fix_point(str_):
    left_str, right_str = str_.split('.')
    left_str = left_str.strip() + '.'

    return left_str + right_str

def generate_description_from_side_attributes(df_row, side='left'):
    if side not in ('left', 'right'):
        raise ValueError("Wrong side value")

    return ' '.join(
        [str(df_row[key]) for key in df_row.keys() if
            key.startswith(f'{side}_') and
            df_row[key] is not np.nan and
            key not in (f'{side}_id', f'{side}_description')])

def split_left_and_right_word_prefixes(group_df):
    encoded_left_desc = group_df[group_df['column'].str.startswith('left_')]['word_prefix'].to_list()
    encoded_right_desc = group_df[group_df['column'].str.startswith('right_')]['word_prefix'].to_list()
    return pd.Series([[encoded_left_desc, encoded_right_desc]], index=['encoded_descs'])

counterfactual = neg_ev_expl.counterfactuals_plotting_data.iloc[[0]]
counterfactual['left_Song_Name'] = counterfactual['left_Song_Name'].apply(fix_point)
counterfactual['right_Song_Name'] = counterfactual['right_Song_Name'].apply(fix_point)
# display(counterfactual)

tokens_removed = [
    ['A00_dayum', 'A03_sarah', 'D02_urban', 'D03_cowboy', 'D04_honky', 'D05_tonk'],
    ['I00_this', 'I01_is', 'I02_how', 'I03_we', 'I07_bryan']
]
counterfactual.at[0, 'tokens_removed'] = tokens_removed
counterfactual['left_description'] = counterfactual.apply(generate_description_from_side_attributes, axis=1)
counterfactual['right_description'] = counterfactual.apply(generate_description_from_side_attributes, side='right', axis=1)
# display(counterfactual)

from copy import deepcopy
neg_eval_expl = deepcopy(neg_ev_expl)

# display(neg_ev_expl_fake.variable_mapper.encode_elements(counterfactual))

encoded_descriptions = neg_eval_expl.variable_mapper.encode_elements(counterfactual).groupby('id').apply(
    split_left_and_right_word_prefixes)

# display(encoded_descriptions.values)
# display(encoded_descriptions['encoded_descs'])

counterfactual.at[0, 'encoded_descs'] = encoded_descriptions['encoded_descs'].iloc[0]

# display(counterfactual)

neg_eval_expl.counterfactuals_plotting_data = counterfactual
HTML(neg_eval_expl.plot_counterfactual(beautify_table=False, align_left=True))

In [ ]:
# mettere giù idea di video: AIUTO
# mettere giù tabella su più righe se possibile ricreare la tabella dei positivi: FATTO
# provare su quick_start_wym a caricare iTunes-Amazon e fare i grafici: NON VA :(

---



In [ ]:
neg_ev_expl.counterfactual_examples[(neg_ev_expl.counterfactual_examples['start_pred'] > 0.5) & (neg_ev_expl.counterfactual_examples['start_pred'] < 1)]

In [ ]:
neg_ev_expl.counterfactuals_plotting_data

In [ ]:
# unire il punto al feat. e rivalutare, se diventa un esempio decente abbiamo il nostro counterfactual negativo
best_counterfactual_id = neg_ev_expl.best_counterfactuals.iloc[0].id
best_no_match = neg_ev_expl.dataset[neg_ev_expl.dataset.id == best_counterfactual_id]
display(best_no_match)

In [ ]:
def fix_point(str_):
    left_str, right_str = str_.split('.')
    left_str = left_str.strip() + '.'

    return left_str + right_str

# best_no_match['left_Song_Name'] = best_no_match['left_Song_Name'].apply(fix_point)
# best_no_match['right_Song_Name'] = best_no_match['right_Song_Name'].apply(fix_point)
best_no_match['left_Song_Name'] = ''
best_no_match['right_Song_Name'] = ''

display(best_no_match)

In [ ]:
counterfactual_impacts = neg_ev_expl.impacts_df[neg_ev_expl.impacts_df['id'] == best_counterfactual_id]
counterfactual_impacts = counterfactual_impacts[(counterfactual_impacts['left_attribute'] != 'Song_Name')]
counterfactual_impacts = counterfactual_impacts[(counterfactual_impacts['right_attribute'] != 'Song_Name')]
display(counterfactual_impacts)

In [ ]:
edited_dataset = neg_ev_expl.dataset.copy()
edited_dataset[edited_dataset.id == 75] = best_no_match
# neg_ev_expl.update_settings(dataset=edited_dataset, impacts_df=counterfactual_impacts)
EvaluateExplanation(edited_dataset, counterfactual_impacts, predict_method=wym_ev.predictor, 
                    evaluate_removing_du=True,
                    recompute_embeddings=True,
                    variable_side='all', fixed_side='all')

neg_ev_expl.evaluate_impacts()

In [ ]:
neg_ev_expl.generate_counterfactual_examples()

In [ ]:
HTML(neg_ev_expl.plot_counterfactual())

In [ ]:
neg_ev_expl.impacts_df

In [ ]:
dataset_with_pred = neg_ev_expl.dataset.copy()
preds_per_id = pd.DataFrame(columns=['id', 'start_pred'])

for index, id_ in enumerate(dataset_with_pred.id.unique()):
    preds_per_id = pd.concat([preds_per_id, pd.DataFrame({'id': id_, 'start_pred': neg_ev_expl.start_pred[index]}, index=[0])], ignore_index=True)

dataset_with_pred = dataset_with_pred.merge(preds_per_id, on='id')
dataset_with_pred = dataset_with_pred.sort_values('start_pred', ascending=False)
display(dataset_with_pred)

In [ ]:
low_pred_df = dataset_with_pred[dataset_with_pred['start_pred'] > 0.5]
low_pred_df = low_pred_df.drop(columns=['start_pred']).sort_index()
low_pred_impacts_df = neg_ev_expl.impacts_df[neg_ev_expl.impacts_df.id.isin(low_pred_df.id)]
display(low_pred_impacts_df)

In [ ]:
low_pred_df = dataset_with_pred[dataset_with_pred['start_pred'] > 0.5]
low_pred_df = low_pred_df.drop(columns=['start_pred']).sort_index()
low_pred_impacts_df = neg_ev_expl.impacts_df[neg_ev_expl.impacts_df.id.isin(low_pred_df.id)]

# in una versione tolgo nome, in una nome album
# versione 1

def delete_artist_name(row_):
    if row_['left_attribute'] == 'Artist_Name':
    # if row_['left_attribute'] == 'left_Album_Name':
        row_['left_word'] = ''

    return row_

low_pred_df_no_name = low_pred_df.copy()
low_pred_df_no_name['Artist_Name'] = [''] * low_pred_df_no_name.shape[0]
# low_pred_df_no_name['left_Album_Name'] = [''] * low_pred_df_no_name.shape[0]

low_pred_impacts_df_no_name = low_pred_impacts_df.apply(delete_artist_name, axis=1)

neg_ev_expl.update_settings(dataset=low_pred_df_no_name, impacts_df=low_pred_impacts_df_no_name)
neg_ev_expl.evaluate_impacts()

# dataset_with_pred[dataset_with_pred.id == 102]
# neg_ev_expl.impacts_df[neg_ev_expl.impacts_df.id == 102]

In [ ]:
neg_ev_expl.generate_counterfactual_examples()

In [ ]:
neg_ev_expl.res_df

In [ ]:
def generate_description_from_side_attributes(df_row, side='left'):
    if side not in ('left', 'right'):
        raise ValueError("Wrong side value")

    return ' '.join(
        [str(df_row[key]) for key in df_row.keys() if
            key.startswith(f'{side}_') and df_row[key] is not np.nan and key not in (f'{side}_id', f'{side}_description')])

def split_left_and_right_word_prefixes(group_df):
    encoded_left_desc = group_df[group_df['column'].str.startswith('left_')]['word_prefix'].to_list()
    encoded_right_desc = group_df[group_df['column'].str.startswith('right_')]['word_prefix'].to_list()
    return pd.Series([[encoded_left_desc, encoded_right_desc]], index=['encoded_descs'])

In [ ]:
tmp = neg_ev_expl.res_df.merge(neg_ev_expl.dataset, on='id')
tmp = tmp.loc[tmp.groupby(['id'])['num_tokens'].idxmax()]

display(tmp.sort_values('new_pred', ascending=False))
# display(counterfactuals_plotting_data.sort_values('new_pred', ascending=False))

In [ ]:
tmp[tmp['new_pred'] < 0.5]

In [ ]:
cpd = tmp.copy()
cpd['left_Song_Name'] = tmp['right_Song_Name']
# cpd['left_Album_Name'] = tmp['right_Album_Name']

cpd['left_description'] = cpd.apply(generate_description_from_side_attributes, axis=1)
cpd['right_description'] = cpd.apply(generate_description_from_side_attributes, side='right', axis=1)
encoded_descriptions = neg_ev_expl.variable_mapper.encode_elements(cpd).groupby('id').apply(
            split_left_and_right_word_prefixes)

cpd = cpd.merge(encoded_descriptions, on='id').sort_values('new_pred', ascending=False)
display(cpd)

In [ ]:
neg_ev_expl.counterfactuals_plotting_data = cpd[0:10]
HTML(neg_ev_expl.plot_counterfactual())

# Update sides - remove only on the left

In [ ]:
utility = True #@param ["True", "False", "\"degradation\"", "\"single_units\"", "\"incremental\"", "\"all\"", "\"AOPC\"", "\"sufficiency\""] {type:"raw"}
# number of casual perturbations 
k = 10 #@param {type:"slider", min:1, max:10, step:1}
wym_ev.update_settings(variable_side='left', fixed_side='right', evaluate_removing_du=False)
pos_res_df, pos_ev_expl, neg_res_df, neg_ev_expl = wym_ev.evaluate_single_df(utility=utility, k=k)

display(pos_res_df)

## Show positive examples

In [ ]:
pos_ev_expl.generate_counterfactual_examples()
display(pos_ev_expl.counterfactual_examples)
display(pos_ev_expl.best_counterfactuals)
display(pos_ev_expl.counterfactuals_plotting_data)

In [ ]:
HTML(pos_ev_expl.plot_counterfactual())

## Show negative examples

In [ ]:
neg_ev_expl.generate_counterfactual_examples()
display(neg_ev_expl.counterfactual_examples)
display(neg_ev_expl.best_counterfactuals)
display(neg_ev_expl.counterfactuals_plotting_data)

In [ ]:
HTML(neg_ev_expl.plot_counterfactual())

# Update sides - remove only on the right

In [ ]:
utility = True #@param ["True", "False", "\"degradation\"", "\"single_units\"", "\"incremental\"", "\"all\"", "\"AOPC\"", "\"sufficiency\""] {type:"raw"}
# number of casual perturbations 
k = 10 #@param {type:"slider", min:1, max:10, step:1}
wym_ev.update_settings(variable_side='right', fixed_side='left', evaluate_removing_du=False)
pos_res_df, pos_ev_expl, neg_res_df, neg_ev_expl = wym_ev.evaluate_single_df(utility=utility, k=k)

display(pos_res_df)

## Show positive examples

In [ ]:
pos_ev_expl.generate_counterfactual_examples()
display(pos_ev_expl.counterfactual_examples)
display(pos_ev_expl.best_counterfactuals)
display(pos_ev_expl.counterfactuals_plotting_data)

In [ ]:
HTML(pos_ev_expl.plot_counterfactual())

## Show negative examples

In [ ]:
neg_ev_expl.generate_counterfactual_examples()
display(neg_ev_expl.counterfactual_examples)
display(neg_ev_expl.best_counterfactuals)
display(neg_ev_expl.counterfactuals_plotting_data)

In [ ]:
HTML(neg_ev_expl.plot_counterfactual())

# --- To Fix ---

# Passing a new DataFrame

In [ ]:
fifty_percent = int(round(wym_ev.dataset.shape[0] / 100 * 50, 0))
new_df = wym_ev.dataset.iloc[:fifty_percent]

In [ ]:
wym_ev.update_settings(dataset_df=new_df)  # TODO: fix this with Andrea, init_routine needs to be edited

## Single description removal

In [ ]:
# remove tokens to generate counterfactual examples

utility = 'all' # @param ["'AOPC'", "'sufficiency'", "'degradation'", "'single_units'", "'incremental'", "'all'", "False", "True"] {type:"raw"}
# number of casual perturbations 
k = 10 #@param {type:"slider", min:1, max:10, step:1}
wym_ev.update_settings(variable_side='right', fixed_side='left', evaluate_removing_du=False)
res_incr_df, ev_expl = wym_ev.evaluate_single_df(utility=utility, k=k)

display(res_incr_df)

In [ ]:
counterfactual_incr = res_incr_df[res_incr_df['detected_delta'] >= 0.5]
print(f"Found {len(counterfactual_incr)} counterfactual examples")
display(counterfactual_incr)

best_counterfactuals = counterfactual_incr.loc[counterfactual_incr.groupby(['id'])['num_tokens'].idxmax()]
print(f"Selected {len(best_counterfactuals)} best counterfactual examples.")
display(best_counterfactuals)

In [ ]:
def split_encoded_left_and_right(group_df, side='left'):
    if side not in ('all', 'left', 'right'):
        raise ValueError(f'Invalid argument side="{side}". Accepted values are "all", "left" and "right".')

    if side == 'all':
        encoded_left_desc = group_df[group_df['column'].str.startswith('left_')]['word_prefix'].to_list()
        encoded_right_desc = group_df[group_df['column'].str.startswith('right_')]['word_prefix'].to_list()

        encoded_desc = pd.Series([[encoded_left_desc, encoded_right_desc]], index=['encoded_descs']) 

    else:
        display(group_df[group_df['column'].str.startswith(f'{side}_')])
        encoded_desc = group_df[group_df['column'].str.startswith(f'{side}_')]['word_prefix'].to_list()

        if side == 'left':
            encoded_desc = pd.Series([[encoded_desc, list()]], index=['encoded_descs'])

        else:  # right side
            encoded_desc = pd.Series([[list(), encoded_desc]], index=['encoded_descs'])

    return encoded_desc 

def generate_description(df_row, side='left'):
    if side not in ('left', 'right'):
        print("Wrong side value")
        return None

    return ' '.join([str(df_row[key]) for key in df_row.keys() if key.startswith(f'{side}_') and key != f'{side}_id' and df_row[key] is not np.nan])

In [ ]:
landmark_plotting_data = best_counterfactuals.merge(wym_ev.desc_to_eval_df, on='id')
landmark_plotting_data['left_description'] = landmark_plotting_data.apply(generate_description, axis=1)
landmark_plotting_data['right_description'] = landmark_plotting_data.apply(generate_description, side='right', axis=1)

encoded_descriptions = ev_expl.mapper_variable.encode_elements(wym_ev.desc_to_eval_df).groupby('id').apply(split_encoded_left_and_right, side='left')

landmark_plotting_data = landmark_plotting_data.merge(encoded_descriptions, on='id')
display(landmark_plotting_data.loc[[0], ['tokens_removed', 'left_description']])
display(landmark_plotting_data.loc[[0], ['encoded_descs']].values[0][0])
display(landmark_plotting_data)

In [ ]:
display(HTML(PlotExplanation.plot_counterfactual(landmark_plotting_data)))

# Extra - experiments

In [ ]:
res_df

In [ ]:
sigmoid = lambda x : 1 / (1+ np.exp(- np.sum(x)))
inv_sigmoid = lambda p : np.log(p/(1-p))

sigmoid(token_contrib.sum() + lr_bias)

In [ ]:
pred_before_sig = res_df['start_pred'].apply(inv_sigmoid)

expected_pred_before_sig = pred_before_sig - res_df['expected_delta']

expected_pred = expected_pred_before_sig.apply(sigmoid)

res_df['expected_delta_after_sigmoid'] = res_df['start_pred'] - expected_pred

In [ ]:
res_df.describe()

In [ ]:
ev.tmp_dataset

## Token contribution studies

In [ ]:
from typing import Union

In [ ]:
start_pred = predictor(match_df)
start_pred

In [ ]:
lr_bias = routine.model['LR'].intercept_[0]
token_contrib = matches_du_prefix_impacts.query('id == 1')['token_contribution'].values  # numpy array
display(token_contrib)
display(lr_bias)

In [ ]:
def convert_token_contribution_to_impact(du_contributions: Union[pd.Series, np.ndarray], bias: float, return_bias: bool = False, return_array: bool = False) -> Union[pd.Series, np.ndarray]:
    e = np.exp
    log = np.log
    # softmax = lambda x: e(x) / sum(e(x))
    # linear = lambda x: x / sum(x)
    sigmoid_scaled = lambda x: ((1 / (1 + e(-x))) - 0.5)
    scale_function = lambda x: sigmoid_scaled(x) / sum(sigmoid_scaled(x))

    if type(du_contributions) is pd.Series:
        du_contributions = du_contributions.values

    all_contrib = np.append(du_contributions, bias)
    impacts = np.zeros_like(all_contrib)
    
    pos_mask = all_contrib >= 0

    pos_contrib = all_contrib[pos_mask]
    neg_contrib = all_contrib[~pos_mask] 
    
    pos_sum = pos_contrib.sum()
    neg_sum = neg_contrib.sum()
    
    tot_sum = pos_sum + neg_sum

    pred_pos = ((e(tot_sum) * pos_sum) /
        ((e(tot_sum) + 1) * tot_sum))

    pred_neg = ((e(tot_sum) * neg_sum) /
        ((e(tot_sum) + 1) * tot_sum))
    
    impacts[pos_mask] = scale_function(pos_contrib) * pred_pos
    impacts[~pos_mask] = scale_function(neg_contrib) * pred_neg

    impacts, bias_impact = impacts[:-1], impacts[-1]

    if not return_array:
        impacts = pd.Series(impacts)

    if return_bias:
        return impacts, bias_impact
    else:
        return impacts

impacts, bias_impact = convert_token_contribution_to_impact(token_contrib, lr_bias, return_bias=True)
print(f'Impacts: \n{impacts}\nPred: {impacts.sum() + bias_impact}\nstart_pred: {start_pred[0]}')

In [ ]:
matches_du

In [ ]:
compact = False # @param {type:"boolean"}
matches_du_test = matches_du.copy()
du_by_id = matches_du.groupby('id')
du_by_id = du_by_id['token_contribution'].apply(convert_token_contribution_to_impact, lr_bias)
matches_du_test['impact'] = du_by_id.values


# if not compact:
#     du_by_id = matches_du.groupby('id')
#     du_by_id = du_by_id['token_contribution'].apply(convert_token_contribution_to_impact, lr_bias).unstack()

#     for id_ in matches_du_test['id'].unique():
#         for row_idx, impacts_idx in zip(matches_du_test[matches_du_test['id'] == id_].index, du_by_id.loc[id_].index):
#             matches_du_test.at[row_idx, 'impact'] = du_by_id.loc[id_][impacts_idx]

# # in alternativa
# else:
#     for id in matches_du_test.id.unique():
#         matches_du_test.loc[matches_du_test['id'] == id, 'impact'] = \
#         matches_du[matches_du['id'] == id][['token_contribution']].apply(
#             convert_token_contribution_to_impact, args=(lr_bias,)).values

display(matches_du_test)

matches_du_new_impact = matches_du_test

In [ ]:
matches_du_new_impact['impact'].hist(bins=100)
matches_du_new_impact['impact'].describe()

In [ ]:
du_by_id = matches_du.groupby('id')
du_by_id = du_by_id['token_contribution'].apply(convert_token_contribution_to_impact, lr_bias)
display(du_by_id)
display(du_by_id.values)
matches_du_test_3 = matches_du.copy()
matches_du_test_3['impact'] = du_by_id.values
display(matches_du_test_3)
all(matches_du_test_3['impact'] == matches_du_test['impact'])

In [ ]:
matches_du_test_1 = matches_du.copy()
matches_du_test_2 = matches_du.copy()

matches_du_test_1['impact'] = [0] * len(matches_du_test_1)
matches_du_test_2['impact'] = [0] * len(matches_du_test_2)
du_by_id = matches_du.groupby('id')
du_by_id = du_by_id['token_contribution'].apply(convert_token_contribution_to_impact, lr_bias).unstack()
for id_ in matches_du_test_1['id'].unique():
    for row_idx, impacts_idx in zip(matches_du_test_1[matches_du_test_1['id'] == id_].index, du_by_id.loc[id_].index):
        matches_du_test_1.at[row_idx, 'impact'] = du_by_id.loc[id_][impacts_idx]


for id in matches_du_test_2.id.unique():
    matches_du_test_2.loc[matches_du_test_2['id'] == id, 'impact'] = matches_du_test_2[matches_du_test_2['id'] == id][['token_contribution']].apply(convert_token_contribution_to_impact, args=(lr_bias,)).values

False in (matches_du_test_1['impact'] == matches_du_test_2['impact'])

In [ ]:
display(matches_du)
du_by_id = matches_du.groupby('id')
du_by_id['token_contribution'].apply(convert_token_contribution_to_impact, lr_bias).unstack()  # .reset_index()
#matches_du['impact'] = [0] * len(matches_du)
# for id_, impacts in du_by_id:


In [ ]:
matches_du_new_impact_prefix = append_prefix(match_df, matches_du_new_impact, decision_unit_view=decision_unit_view,
                                        exclude_attrs=exclude_attrs)

# evaluate WYM evaluation with Landmark
ev = EvaluateExplanation(match_df, matches_du_new_impact_prefix, predict_method=predictor,
                            exclude_attrs=exclude_attrs, percentage=.25, num_round=3,
                            decision_unit_view=decision_unit_view,
                            remove_decision_unit_only=remove_decision_unit_only)

variable_side = fixed_side = 'all'
utility = True  # change class
res: list = ev.evaluate_impacts(match_df, matches_du_new_impact_prefix,
                                variable_side, fixed_side, 
                                add_before_perturbation=None,
                                add_after_perturbation=None, 
                                utility=utility, k=k)

res_df = pd.DataFrame(res)
# res_df['conf'] = conf_name
res_df['error'] = res_df.expected_delta - res_df.detected_delta
display(res_df)

In [ ]:
counterfactual = res_df[(res_df['detected_delta'] >= 0.5) & ~(res_df['comb_name'].isin(['all_opposite']))]
print(f"Found {len(counterfactual)} counterfactual examples out of {len(res_df)}")
display(counterfactual)

In [ ]:
utility = 'single_units'  # change class
res_single_units: list = ev.evaluate_impacts(match_df, matches_du_new_impact_prefix,
                                variable_side, fixed_side, 
                                add_before_perturbation=None,
                                add_after_perturbation=None, 
                                utility=utility, k=k)

res_su_df = pd.DataFrame(res_single_units)
# res_df['conf'] = conf_name
res_su_df['error'] = res_su_df.expected_delta - res_su_df.detected_delta
display(res_su_df)

In [ ]:
counterfactual_su = res_su_df[res_su_df['detected_delta'] >= 0.5]
print(f"Found {len(counterfactual_su)} counterfactual examples")
display(counterfactual_su)